# Pull Request Analysis

In [2]:
import psycopg2
import pandas as pd 
# from sqlalchemy.types import Integer, Text, String, DateTime
import sqlalchemy as s
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = s.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [3]:
repo_id_list = pd.DataFrame()
myQuery = f"""
SELECT repo_group_id, repo_id, repo_name from repo;
    """
repo_id_list = pd.read_sql_query(myQuery, con=engine)
print(repo_id_list)

      repo_group_id  repo_id                           repo_name
0             25170    26687            distributed-tracing-demo
1             25170    26720        pa-services-execution-method
2             25179    27493  docs-boomi-atomsphere-int-services
3             25176    27178                    Out-Of-Queue-Bot
4             25177    27191        tcserver-spring-boot-samples
...             ...      ...                                 ...
2404          25171    26973                              fiscal
2405          25179    27274                                a11y
2406          25164    26130                philips-hue-resource
2407          25179    27399           pcf-bosh-terraforming-gcp
2408          25155    25541                rabbitmq-trust-store

[2409 rows x 3 columns]


## Pull Request Filter

In [2]:
## List of repository IDs for the report
repo_dict = {26214, 26215,26216, 26220, 26219, 25158, 26217, 26218}

In [4]:
contributor = pd.DataFrame()
cquery = f"""
SELECT
	* 
FROM
	( SELECT * FROM repos ) AS repolist
	LEFT JOIN (
	SELECT
		RH.*,
		`All`.*,
		RH.RH_commits / `All`.all_commits AS commit_pct,
		RH.RH_added / `All`.all_added AS added_pct,
		RH.RH_deleted / `All`.all_deleted AS deleted_pct,
		RH.RH_whitespace / `All`.all_whitespace AS whitespace_pct 
	FROM
		(
		SELECT
			b.id AS all_repos_id,
			count( c.COMMIT ) AS all_commits,
			sum( c.added ) AS all_added,
			sum( c.removed ) AS all_deleted,
			sum( c.whitespace ) AS all_whitespace 
		FROM
			repo_groups a,
			repo b,
			commits c 
		WHERE
			a.id = b.projects_id 
			AND b.id = c.repos_id 
		GROUP BY
			a.id,
			b.id 
		ORDER BY
			a.id,
			b.id 
		) AS `All`
		LEFT JOIN (
		SELECT
			b.id AS RH_repos_id,
			count( c.COMMIT ) AS RH_commits,
			sum( c.added ) AS RH_added,
			sum( c.removed ) AS RH_deleted,
			sum( c.whitespace ) AS RH_whitespace 
		FROM
			projects a,
			repos b,
			analysis_data c 
		WHERE
			a.id = b.projects_id 
			AND b.id = c.repos_id 
			AND c.author_email LIKE '%pivotal%' 
		GROUP BY
			a.id,
			b.id 
		ORDER BY
			a.id,
			b.id 
		) AS RH ON `All`.all_repos_id = RH.RH_repos_id 
	ORDER BY
		`All`.all_repos_id,
		RH.RH_repos_id 
	) AS allcommits ON repolist.id = allcommits.all_repos_id 
ORDER BY
	allcommits.all_repos_id;
""" 

contributor = pd.read_sql_query(cquery, con=engine)
print(conributor)

TypeError: 'dict' object does not support indexing